---

# Очистка данных

***Мусор на входе модели $\to$ мусор на выходе.***

Ни одна модель машинного обучения не выдаст осмысленных результатов, если вы предоставите ей сырые данные. После формирования выборки данных их необходимо очистить.

In [68]:
#main
import pandas as pd
import numpy as np

#plot
from matplotlib import pyplot as plt
import seaborn as sns

#plot
from matplotlib import pyplot as plt
import seaborn as sns

#
from tqdm import tqdm 
import re

pd.set_option("display.max_columns", 300)


# Имеем :
PATH_TRAIN = './data/train_sber.csv'
PATH_MACRO = './data/macro.csv'

**У нас есть достаточно большой DataSet с кучей признаков много пропусков и мусорных значений. Будем обрабатывать!**

In [108]:
df = pd.read_csv(PATH_TRAIN)
df_makro = pd.read_csv(PATH_MACRO)
df_y = df['price_doc']
df.drop(columns='price_doc', axis=1, inplace=True)

**Для заполнения ошибочных значений надо вникнуть в суть признаков и определить их целесообразность  
В папке data/ файл data_dictionary.txt содержит описание всех признаков, посмотрим их**

В первую очередь можно заметить что первые 13 признаков это данные самой транзакции, а далее это различные характеристики района 

In [109]:
df.iloc[:, 0:13].head()

,id,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,state,product_type,sub_area
0,1,2011-08-20,43,27.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,Investment,Bibirevo
1,2,2011-08-23,34,19.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,Investment,Nagatinskij Zaton
2,3,2011-08-27,43,29.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,Investment,Tekstil'shhiki
3,4,2011-09-01,89,50.0,9.0,NaN,NaN,NaN,NaN,NaN,NaN,Investment,Mitino
4,5,2011-09-05,77,77.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,Investment,Basmannoe


Это важные признаки назавем их:

Признаки I очереди
---

id timestamp full_sq life_sq floor max_floor material build_year num_room kitch_sq state product_type sub_area

Для построения модели они болие важны

In [110]:
df_x1 = df.iloc[:, 0:13]

In [111]:
df[df['sub_area'] == 'Mitino'].iloc[:, 12:]

,sub_area,area_m,raion_popul,green_zone_part,indust_part,children_preschool,preschool_quota,preschool_education_centers_raion,children_school,school_quota,school_education_centers_raion,school_education_centers_top_20_raion,hospital_beds_raion,healthcare_centers_raion,university_top_20_raion,sport_objects_raion,additional_education_raion,culture_objects_top_25,culture_objects_top_25_raion,shopping_centers_raion,office_raion,thermal_power_plant_raion,incineration_raion,oil_chemistry_raion,radiation_raion,railroad_terminal_raion,big_market_raion,nuclear_reactor_raion,detention_facility_raion,full_all,male_f,female_f,young_all,young_male,young_female,work_all,work_male,work_female,ekder_all,ekder_male,ekder_female,0_6_all,0_6_male,0_6_female,7_14_all,7_14_male,7_14_female,0_17_all,0_17_male,0_17_female,16_29_all,16_29_male,16_29_female,0_13_all,0_13_male,0_13_female,raion_build_count_with_material_info,build_count_block,build_count_wood,build_count_frame,build_count_brick,build_count_monolith,build_count_panel,build_count_foam,build_count_slag,build_count_mix,raion_build_count_with_builddate_info,build_count_before_1920,build_count_1921-1945,build_count_1946-1970,build_count_1971-1995,build_count_after_1995,ID_metro,metro_min_avto,metro_km_avto,metro_min_walk,metro_km_walk,kindergarten_km,school_km,park_km,green_zone_km,industrial_km,water_treatment_km,cemetery_km,incineration_km,railroad_station_walk_km,railroad_station_walk_min,ID_railroad_station_walk,railroad_station_avto_km,railroad_station_avto_min,ID_railroad_station_avto,public_transport_station_km,public_transport_station_min_walk,water_km,water_1line,mkad_km,ttk_km,sadovoe_km,bulvar_ring_km,kremlin_km,big_road1_km,ID_big_road1,big_road1_1line,big_road2_km,ID_big_road2,railroad_km,railroad_1line,zd_vokzaly_avto_km,ID_railroad_terminal,bus_terminal_avto_km,ID_bus_terminal,oil_chemistry_km,nuclear_reactor_km,radiation_km,power_transmission_line_km,thermal_power_plant_km,ts_km,big_market_km,market_shop_km,fitness_km,swim_pool_km,ice_rink_km,stadium_km,basketball_km,hospice_morgue_km,detention_facility_km,public_healthcare_km,university_km,workplaces_km,shopping_centers_km,office_km,additional_education_km,preschool_km,big_church_km,church_synagogue_km,mosque_km,theater_km,museum_km,exhibition_km,catering_km,ecology,green_part_500,prom_part_500,office_count_500,office_sqm_500,trc_count_500,trc_sqm_500,cafe_count_500,cafe_sum_500_min_price_avg,cafe_sum_500_max_price_avg,cafe_avg_price_500,cafe_count_500_na_price,cafe_count_500_price_500,cafe_count_500_price_1000,cafe_count_500_price_1500,cafe_count_500_price_2500,cafe_count_500_price_4000,cafe_count_500_price_high,big_church_count_500,church_count_500,mosque_count_500,leisure_count_500,sport_count_500,market_count_500,green_part_1000,prom_part_1000,office_count_1000,office_sqm_1000,trc_count_1000,trc_sqm_1000,cafe_count_1000,cafe_sum_1000_min_price_avg,cafe_sum_1000_max_price_avg,cafe_avg_price_1000,cafe_count_1000_na_price,cafe_count_1000_price_500,cafe_count_1000_price_1000,cafe_count_1000_price_1500,cafe_count_1000_price_2500,cafe_count_1000_price_4000,cafe_count_1000_price_high,big_church_count_1000,church_count_1000,mosque_count_1000,leisure_count_1000,sport_count_1000,market_count_1000,green_part_1500,prom_part_1500,office_count_1500,office_sqm_1500,trc_count_1500,trc_sqm_1500,cafe_count_1500,cafe_sum_1500_min_price_avg,cafe_sum_1500_max_price_avg,cafe_avg_price_1500,cafe_count_1500_na_price,cafe_count_1500_price_500,cafe_count_1500_price_1000,cafe_count_1500_price_1500,cafe_count_1500_price_2500,cafe_count_1500_price_4000,cafe_count_1500_price_high,big_church_count_1500,church_count_1500,mosque_count_1500,leisure_count_1500,sport_count_1500,market_count_1500,green_part_2000,prom_part_2000,office_count_2000,office_sqm_2000,trc_count_2000,trc_sqm_2000,cafe_count_2000,cafe_sum_2000_min_price_avg,cafe_sum_2000_max_price_avg,cafe_avg_price_2000,cafe_count_2000_na_price,cafe_count_2000_price_500,cafe_count_2000_price_1000,ca

___

Можно заметить что от sub_area до ID_metro признаки отнасительно района и все индетичны: выделим это в отдельный dataset И поработаем с ним позже

In [112]:
df_sub_area = df.loc[:,'sub_area': 'build_count_after_1995']
df.drop(columns=df.loc[:,'area_m': 'build_count_after_1995'].columns, axis=1, inplace=True)

In [54]:
df.loc[:, 'ID_metro':].head()

,ID_metro,metro_min_avto,metro_km_avto,metro_min_walk,metro_km_walk,kindergarten_km,school_km,park_km,green_zone_km,industrial_km,water_treatment_km,cemetery_km,incineration_km,railroad_station_walk_km,railroad_station_walk_min,ID_railroad_station_walk,railroad_station_avto_km,railroad_station_avto_min,ID_railroad_station_avto,public_transport_station_km,public_transport_station_min_walk,water_km,water_1line,mkad_km,ttk_km,sadovoe_km,bulvar_ring_km,kremlin_km,big_road1_km,ID_big_road1,big_road1_1line,big_road2_km,ID_big_road2,railroad_km,railroad_1line,zd_vokzaly_avto_km,ID_railroad_terminal,bus_terminal_avto_km,ID_bus_terminal,oil_chemistry_km,nuclear_reactor_km,radiation_km,power_transmission_line_km,thermal_power_plant_km,ts_km,big_market_km,market_shop_km,fitness_km,swim_pool_km,ice_rink_km,stadium_km,basketball_km,hospice_morgue_km,detention_facility_km,public_healthcare_km,university_km,workplaces_km,shopping_centers_km,office_km,additional_education_km,preschool_km,big_church_km,church_synagogue_km,mosque_km,theater_km,museum_km,exhibition_km,catering_km,ecology,green_part_500,prom_part_500,office_count_500,office_sqm_500,trc_count_500,trc_sqm_500,cafe_count_500,cafe_sum_500_min_price_avg,cafe_sum_500_max_price_avg,cafe_avg_price_500,cafe_count_500_na_price,cafe_count_500_price_500,cafe_count_500_price_1000,cafe_count_500_price_1500,cafe_count_500_price_2500,cafe_count_500_price_4000,cafe_count_500_price_high,big_church_count_500,church_count_500,mosque_count_500,leisure_count_500,sport_count_500,market_count_500,green_part_1000,prom_part_1000,office_count_1000,office_sqm_1000,trc_count_1000,trc_sqm_1000,cafe_count_1000,cafe_sum_1000_min_price_avg,cafe_sum_1000_max_price_avg,cafe_avg_price_1000,cafe_count_1000_na_price,cafe_count_1000_price_500,cafe_count_1000_price_1000,cafe_count_1000_price_1500,cafe_count_1000_price_2500,cafe_count_1000_price_4000,cafe_count_1000_price_high,big_church_count_1000,church_count_1000,mosque_count_1000,leisure_count_1000,sport_count_1000,market_count_1000,green_part_1500,prom_part_1500,office_count_1500,office_sqm_1500,trc_count_1500,trc_sqm_1500,cafe_count_1500,cafe_sum_1500_min_price_avg,cafe_sum_1500_max_price_avg,cafe_avg_price_1500,cafe_count_1500_na_price,cafe_count_1500_price_500,cafe_count_1500_price_1000,cafe_count_1500_price_1500,cafe_count_1500_price_2500,cafe_count_1500_price_4000,cafe_count_1500_price_high,big_church_count_1500,church_count_1500,mosque_count_1500,leisure_count_1500,sport_count_1500,market_count_1500,green_part_2000,prom_part_2000,office_count_2000,office_sqm_2000,trc_count_2000,trc_sqm_2000,cafe_count_2000,cafe_sum_2000_min_price_avg,cafe_sum_2000_max_price_avg,cafe_avg_price_2000,cafe_count_2000_na_price,cafe_count_2000_price_500,cafe_count_2000_price_1000,cafe_count_2000_price_1500,cafe_count_2000_price_2500,cafe_count_2000_price_4000,cafe_count_2000_price_high,big_church_count_2000,church_count_2000,mosque_count_2000,leisure_count_2000,sport_count_2000,market_count_2000,green_part_3000,prom_part_3000,office_count_3000,office_sqm_3000,trc_count_3000,trc_sqm_3000,cafe_count_3000,cafe_sum_3000_min_price_avg,cafe_sum_3000_max_price_avg,cafe_avg_price_3000,cafe_count_3000_na_price,cafe_count_3000_price_500,cafe_count_3000_price_1000,cafe_count_3000_price_1500,cafe_count_3000_price_2500,cafe_count_3000_price_4000,cafe_count_3000_price_high,big_church_count_3000,church_count_3000,mosque_count_3000,leisure_count_3000,sport_count_3000,market_count_3000,green_part_5000,prom_part_5000,office_count_5000,office_sqm_5000,trc_count_5000,trc_sqm_5000,cafe_count_5000,cafe_sum_5000_min_price_avg,cafe_sum_5000_max_price_avg,cafe_avg_price_5000,cafe_count_5000_na_price,cafe_count_5000_price_500,cafe_count_5000_price_1000,cafe_count_5000_price_1500,cafe_count_5000_price_2500,cafe_count_5000_price_4000,cafe_count_5000_price_high,big_church_count_5000,church_count_5000,mosque_count_5000,leisure_count_5000,sport_count_5000,market_count_5000,price_doc
0,1,2.590241,1.131260,13.5751

ID_metro и далие отсечка для болие глубоких признаков выделим это в 

Признаки II очереди
---
---

In [113]:
df_x2 = df.loc[:, 'ID_metro':]

## Итог
### визуального осмотра признаков

Это разделение на несколько dataset - ов

df_y - искомая переменная  
df_x1 - основные признаки  
df_x2 - второстепенные глубокие признаки  
df_sub_area - повторяющиеся данные района  

---
обеденним все действия по разделению dataset в одну функцию ниже

---
## split_df
---

In [167]:
def split_df(df):
    try:
        df_y = df['price_doc']
    except KeyError:
        df_y = None
    
    df_x1 = df.loc[:, :'sub_area']
    df_sub_area = df.loc[:,'sub_area': 'build_count_after_1995'].groupby(by=['sub_area']).mean()
    df_x2 = df.loc[:, 'ID_metro':]
    
    return df_y, df_x1, df_x2, df_sub_area

In [168]:
df = pd.read_csv(PATH_TRAIN)

df_y, df_x1, df_x2, df_sub_area = split_df(df)

---

# Начнем работу с пропусками и выбросами

В этом блоке все необходимые операции над преобразованием признаков сразу пишутся в функции ниже


In [238]:
# Процентный список пропущенных данных
df_x1.isna().mean().round(4)*100

id               0.00
timestamp        0.00
full_sq          0.00
life_sq         20.95
floor            0.55
max_floor       31.41
material        31.41
build_year      44.65
num_room        31.41
kitch_sq        31.41
state           44.50
product_type     0.00
sub_area         0.00
dtype: float64

In [224]:
df_x1.describe()

,id,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,state
count,30471.000000,30471.000000,24088.000000,30304.000000,20899.000000,20899.000000,1.686600e+04,20899.000000,20899.000000,16912.000000
mean,15237.917397,54.214269,34.403271,7.670803,12.558974,1.827121,3.068057e+03,1.909804,6.399301,2.107025
std,8796.501536,38.031487,52.285733,5.319989,6.756550,1.481154,1.543878e+05,0.851805,28.265979,0.880148
min,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000e+00,0.000000,0.000000,1.000000
25%,7620.500000,38.000000,20.000000,3.000000,9.000000,1.000000,1.967000e+03,1.000000,1.000000,1.000000
50%,15238.000000,49.000000,30.000000,6.500000,12.000000,1.000000,1.979000e+03,2.000000,6.000000,2.000000
75%,22855.500000,63.000000,43.000000,11.000000,17.000000,2.000000,2.005000e+03,2.000000,9.000000,3.000000
max,30473.000000,5326.000000,7478.000000,77.000000,117.000000,6.000000,2.005201e+07,19.000000,2014.000000,33.000000


In [283]:
df_x1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30471 entries, 0 to 30470
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            30471 non-null  int64  
 1   timestamp     30471 non-null  object 
 2   full_sq       30471 non-null  int64  
 3   life_sq       24088 non-null  float64
 4   floor         30304 non-null  float64
 5   max_floor     20899 non-null  float64
 6   material      20899 non-null  float64
 7   build_year    16866 non-null  float64
 8   num_room      20899 non-null  float64
 9   kitch_sq      20899 non-null  float64
 10  state         16912 non-null  float64
 11  product_type  30471 non-null  object 
 12  sub_area      30471 non-null  object 
dtypes: float64(8), int64(2), object(3)
memory usage: 3.0+ MB


In [243]:
# Видно желая площадь не может быть больше всей площади, надо обработать ошибку
df_x1[df_x1['full_sq'] < df_x1['life_sq']]

,id,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,state,product_type,sub_area
1084,1085,2012-02-06,44,281.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,Investment,Bibirevo
1188,1189,2012-02-14,10,44.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,Investment,Veshnjaki
1822,1825,2012-04-09,18,38.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Investment,Ljublino
1863,1866,2012-04-11,30,178.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,Investment,Presnenskoe
2009,2012,2012-04-25,10,40.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,Investment,Gol'janovo
4385,4388,2012-10-25,73,426.0,17.0,NaN,NaN,NaN,NaN,NaN,NaN,OwnerOccupier,Vojkovskoe
6336,6339,2013-03-06,37,191.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,Investment,Kosino-Uhtomskoe
6531,6534,2013-03-15,80,88.0,9.0,NaN,NaN,NaN,NaN,NaN,NaN,OwnerOccupier,Pokrovskoe Streshnevo
6993,6996,2013-04-03,73,77.0,10.0,NaN,NaN,NaN,NaN,NaN,NaN,OwnerOccupier,Nekrasovka
7208,7211,2013-04-10,31,195.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,Investment,Cheremushki


In [241]:
# ошибочные значения по нижней границы
df_x1[df_x1['full_sq'] < 10]

,id,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,state,product_type,sub_area
1188,1189,2012-02-14,9,44.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,Investment,Veshnjaki
2009,2012,2012-04-25,5,40.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,Investment,Gol'janovo
6112,6115,2013-02-22,6,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,OwnerOccupier,Poselenie Voskresenskoe
11332,11335,2013-10-18,1,40.0,10.0,17.0,1.0,2013.0,1.0,1.0,1.0,OwnerOccupier,Poselenie Shherbinka
16289,16292,2014-03-20,1,1.0,1.0,1.0,4.0,1.0,1.0,1.0,3.0,OwnerOccupier,Poselenie Voskresenskoe
16738,16741,2014-03-31,1,1.0,1.0,1.0,4.0,1.0,1.0,1.0,3.0,OwnerOccupier,Poselenie Voskresenskoe
17194,17197,2014-04-09,1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,OwnerOccupier,Pokrovskoe Streshnevo
17932,17935,2014-04-28,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,OwnerOccupier,Poselenie Voskresenskoe
18600,18603,2014-05-19,1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0,OwnerOccupier,Tverskoe
22171,22174,2014-09-03,1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,OwnerOccupier,Poselenie Vnukovskoe


In [255]:
#167 значений - заполнить!
df_x1[df_x1['floor'].isna()]

,id,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,state,product_type,sub_area
178,179,2011-10-22,98,94.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Investment,Kurkino
187,188,2011-10-24,49,29.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Investment,Nagatinskij Zaton
195,196,2011-10-25,37,23.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Investment,Nagatinskij Zaton
215,216,2011-10-26,45,27.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Investment,Chertanovo Juzhnoe
294,295,2011-11-05,33,20.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Investment,Novogireevo
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9699,9702,2013-08-08,146,146.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,OwnerOccupier,Poselenie Pervomajskoe
9727,9730,2013-08-09,82,82.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,OwnerOccupier,Troickij okrug
9847,9850,2013-08-15,79,41.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,OwnerOccupier,Poselenie Voskresenskoe
9866,9869,2013-08-16,146,146.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,OwnerOccupier,Poselenie Pervomajskoe


In [266]:
# Пропусков много, такой столбец удалить бы но =) заполним ради спортивного интереса
df_x1[df_x1['max_floor'].isna()]

,id,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,state,product_type,sub_area
0,1,2011-08-20,43,27.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,Investment,Bibirevo
1,2,2011-08-23,34,19.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,Investment,Nagatinskij Zaton
2,3,2011-08-27,43,29.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,Investment,Tekstil'shhiki
3,4,2011-09-01,89,50.0,9.0,NaN,NaN,NaN,NaN,NaN,NaN,Investment,Mitino
4,5,2011-09-05,77,77.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,Investment,Basmannoe
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10290,10293,2013-09-05,47,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,OwnerOccupier,Troickij okrug
10304,10307,2013-09-05,65,60.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,OwnerOccupier,Poselenie Voskresenskoe
10328,10331,2013-09-06,37,NaN,14.0,NaN,NaN,NaN,NaN,NaN,NaN,OwnerOccupier,Severnoe Tushino
10389,10392,2013-09-10,38,NaN,15.0,NaN,NaN,NaN,NaN,NaN,NaN,OwnerOccupier,Nekrasovka


In [272]:
# видно, что все пропуски материала зданий индентичны количеству этажей 
(df_x1[df_x1['material'].isna()].index == df_x1[df_x1['max_floor'].isna()].index).sum()

9572

In [273]:
# и также индентичны количеству комнат 
(df_x1[df_x1['material'].isna()].index == df_x1[df_x1['num_room'].isna()].index).sum()

9572

ПОжалую удалим строкии с пропусками max_floor, material, num_room  
но для тестовых значений придется искать усредненые или типовые среднии

In [282]:
# Признак имеет много пропусков а также ошибок, так как трудно его определить НА УДАЛЕНИЕ целиком признак!
df_x1[df_x1['build_year'] < 1000]

,id,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,state,product_type,sub_area
9441,9444,2013-07-25,73,73.0,5.0,22.0,1.0,0.0,3.0,0.0,NaN,OwnerOccupier,Poselenie Vnukovskoe
9620,9623,2013-08-05,65,35.0,7.0,12.0,1.0,1.0,2.0,1.0,1.0,OwnerOccupier,Zapadnoe Degunino
9700,9703,2013-08-08,60,NaN,24.0,24.0,1.0,1.0,2.0,1.0,1.0,OwnerOccupier,Poselenie Vnukovskoe
9745,9748,2013-08-09,39,39.0,10.0,17.0,1.0,1.0,1.0,12.0,1.0,OwnerOccupier,Nekrasovka
9764,9767,2013-08-12,119,NaN,24.0,1.0,1.0,1.0,1.0,1.0,1.0,OwnerOccupier,Mitino
...,...,...,...,...,...,...,...,...,...,...,...,...,...
30394,30397,2015-06-24,52,52.0,14.0,17.0,1.0,1.0,2.0,1.0,1.0,OwnerOccupier,Poselenie Vnukovskoe
30425,30428,2015-06-25,64,NaN,13.0,17.0,1.0,0.0,2.0,1.0,1.0,OwnerOccupier,Solncevo
30429,30432,2015-06-26,42,NaN,7.0,17.0,6.0,0.0,1.0,1.0,1.0,OwnerOccupier,Poselenie Vnukovskoe
30430,30433,2015-06-26,52,NaN,11.0,17.0,1.0,0.0,2.0,1.0,1.0,OwnerOccupier,Poselenie Vnukovskoe


In [385]:
def moustache(s):
    # Принемает series
    # Возращает значения усов нормального распределения
    q75,q25 = np.percentile(s,[75,25])
    iqr = q75 - q25
    return q25 - 1.5*iqr, q75 + 1.5*iqr

In [398]:

def cleaning_x1(df):
    # копия что-бы в эксперементах не испортить dataset
    res = df.copy()

    # уберем ошибочные значения по нижней границы
    res.loc[res['full_sq'] < 10, 'full_sq'] = 10
    res.loc[res['life_sq'] < 10, 'life_sq'] = res.loc[res['life_sq'] < 10, 'full_sq']

    # жилую площадь обработаем просто
    res.loc[res['life_sq'].isna(), 'life_sq'] = res.loc[res['life_sq'].isna(), 'full_sq']

    # Заполним пропуски этажа
    index_nan_floor = list(res[res['floor'].isna()].index)
    for i in tqdm(index_nan_floor, desc='floor'):
        mode_floor = res[res['sub_area'] == res.loc[i]['sub_area']]['max_floor'].mode()[0] // 2
        res.loc[i, 'floor'] = mode_floor

    # Заполнение max_floor
    index_nan_max_floor = list(res[res['max_floor'].isna()].index)
    for i in tqdm(index_nan_max_floor, desc='max_floor'):
        mode_max_floor = res[res['sub_area'] == res.loc[i]['sub_area']]['max_floor'].mode()[0]
        res.loc[i, 'max_floor'] = mode_max_floor
    
    # Пропуски по количеству комнат заполним по похожим площадям
    index_nan_num_room = list(res[res['num_room'].isna()].index)
    sq_one_room = (df_x1[~df_x1['num_room'].isna()]['full_sq'] / 
                   df_x1[~df_x1['num_room'].isna()]['num_room']).median()
    for i in tqdm(index_nan_num_room, desc='num_room'):
        if res.loc[i, 'full_sq'] < sq_one_room:
            res.loc[i, 'num_room'] = 1
        else:
            res.loc[i, 'num_room'] = res.loc[i, 'full_sq'] // sq_one_room
    
    # Удалим по причине мультиколлинеарности и большого количество пропусков
    res.drop(columns=['kitch_sq'], axis=1, inplace=True)
    return res

tmp = cleaning_x1(df_x1)

num_room: 100%|████████████████████████████████████████████████| 9572/9572 [00:05<00:00, 1777.59it/s]


In [399]:
tmp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30471 entries, 0 to 30470
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            30471 non-null  int64  
 1   timestamp     30471 non-null  object 
 2   full_sq       30471 non-null  int64  
 3   life_sq       30471 non-null  float64
 4   floor         30471 non-null  float64
 5   max_floor     30471 non-null  float64
 6   material      20899 non-null  float64
 7   build_year    16866 non-null  float64
 8   num_room      30471 non-null  float64
 9   state         16912 non-null  float64
 10  product_type  30471 non-null  object 
 11  sub_area      30471 non-null  object 
dtypes: float64(7), int64(2), object(3)
memory usage: 2.8+ MB
